In [1]:
import sqlite3
import os
from datetime import datetime

In [3]:
# criando a base
def criar_banco():
    conn = sqlite3.connect('clientes.db')
    cursor = conn.cursor()
    
    # cliente
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS clientes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT NOT NULL,
            telefone TEXT UNIQUE NOT NULL,
            cortes INTEGER DEFAULT 0
        )
    ''')
    
    # datas cortes
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS cortes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            cliente_id INTEGER,
            data TEXT,
            FOREIGN KEY(cliente_id) REFERENCES clientes(id)
        )
    ''')
    
    conn.commit()
    conn.close()

In [4]:
# valido se o numero e valido 
def validar_telefone(telefone):
    return telefone.isdigit()

# consulto se o numero esta cadastrado
def consultar_cliente(telefone):
    conn = sqlite3.connect('clientes.db')
    cursor = conn.cursor()
    cursor.execute("SELECT id, nome, telefone, cortes FROM clientes WHERE telefone = ?", (telefone,))
    resultado = cursor.fetchone()
    conn.close()
    return resultado 

# castrado de novo cliente
def cadastrar_cliente(nome, telefone, cortou, data):
    conn = sqlite3.connect('clientes.db')
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO clientes (nome, telefone, cortes) VALUES (?, ?, ?)", (nome, telefone, 1 if cortou else 0))
        cliente_id = cursor.lastrowid
        if cortou:
            cursor.execute("INSERT INTO cortes (cliente_id, data) VALUES (?, ?)", (cliente_id, data))
        conn.commit()
        print("Cliente cadastrado com sucesso!")
    except sqlite3.IntegrityError:
        print("Erro: Este telefone já está cadastrado.")
    finally:
        conn.close()
        
# atualizo os cortes dos clientes existentes
def atualizar_cortes(cliente_id, cortou, data):
    conn = sqlite3.connect('clientes.db')
    cursor = conn.cursor()
    if cortou:
        cursor.execute("UPDATE clientes SET cortes = cortes + 1 WHERE id = ?", (cliente_id,))
        cursor.execute("INSERT INTO cortes (cliente_id, data) VALUES (?, ?)", (cliente_id, data))
        conn.commit()
        print("Número de cortes atualizado!")
    else:
        print("Nenhum corte registrado.")
    conn.close()

# gerando relatorio em txt
def gerar_relatorio():
    conn = sqlite3.connect('clientes.db')
    cursor = conn.cursor()
    cursor.execute("SELECT nome, telefone, cortes FROM clientes ORDER BY nome ASC")
    clientes = cursor.fetchall()
    conn.close()
    
    with open('clientes.txt', 'w', encoding='utf-8') as arquivo:
        for cliente in clientes:
            arquivo.write(f"Nome: {cliente[0]}, Telefone: {cliente[1]}, Cortes: {cliente[2]}\n")
    
    print("Relatório 'clientes.txt' gerado com sucesso!")

# chamando as funções 
def main():
    criar_banco()
    
    while True:
        print("\n--- Aplicativo de Gerenciamento de Clientes ---")
        print("1. Registrar/Checar Cliente")
        print("2. Gerar Relatório")
        print("3. Sair")
        escolha = input("Escolha uma opção: ")
        
        if escolha == '1':
            telefone = input("Digite o telefone do cliente (apenas números): ")
            if not validar_telefone(telefone):
                print("Telefone inválido! Digite apenas números.")
                continue
            
            cliente = consultar_cliente(telefone)
            if cliente:
                print(f"\nCliente Encontrado:\nNome: {cliente[1]}\nTelefone: {cliente[2]}\nQuantidade de Cortes: {cliente[3]}")
                resposta = input("A pessoa vai cortar o cabelo? (s/n): ").strip().lower()
                if resposta == 's':
                    data = input("Digite a data do corte (DD/MM/AAAA) ou deixe em branco para a data atual: ")
                    if not data:
                        data = datetime.now().strftime("%d/%m/%Y")
                    atualizar_cortes(cliente[0], True, data)
                else:
                    print("Nenhum corte realizado.")
            else:
                print("Cliente não encontrado. Vamos cadastrá-lo.")
                nome = input("Digite o nome do cliente: ").strip()
                if not nome:
                    print("Nome não pode ser vazio.")
                    continue
                cortou_resp = input("A pessoa cortou o cabelo? (s/n): ").strip().lower()
                cortou = cortou_resp == 's'
                data = ""
                if cortou:
                    data = input("Digite a data do corte (DD/MM/AAAA) ou deixe em branco para a data atual: ")
                    if not data:
                        data = datetime.now().strftime("%d/%m/%Y")
                cadastrar_cliente(nome, telefone, cortou, data)
        
        elif escolha == '2':
            gerar_relatorio()
        
        elif escolha == '3':
            print("Saindo do aplicativo. Até mais!")
            break
        
        else:
            print("Opção inválida. Por favor, tente novamente.")

In [17]:
if __name__ == "__main__":
    main()


--- Aplicativo de Gerenciamento de Clientes ---
1. Registrar/Checar Cliente
2. Gerar Relatório
3. Sair
Escolha uma opção: 1
Digite o telefone do cliente (apenas números): 21999
Cliente não encontrado. Vamos cadastrá-lo.
Digite o nome do cliente: Mateus
A pessoa cortou o cabelo? (s/n): s
Digite a data do corte (DD/MM/AAAA) ou deixe em branco para a data atual: 
Cliente cadastrado com sucesso!

--- Aplicativo de Gerenciamento de Clientes ---
1. Registrar/Checar Cliente
2. Gerar Relatório
3. Sair
Escolha uma opção: 1
Digite o telefone do cliente (apenas números): 21999

Cliente Encontrado:
Nome: Mateus
Telefone: 21999
Quantidade de Cortes: 1
A pessoa vai cortar o cabelo? (s/n): n
Nenhum corte realizado.

--- Aplicativo de Gerenciamento de Clientes ---
1. Registrar/Checar Cliente
2. Gerar Relatório
3. Sair
Escolha uma opção: 2
Relatório 'clientes.txt' gerado com sucesso!

--- Aplicativo de Gerenciamento de Clientes ---
1. Registrar/Checar Cliente
2. Gerar Relatório
3. Sair
Escolha uma opç